# Experient 2

In [1]:
import pandas as pd
import numpy as np
import math

In [23]:
base_dir = "Data/"


In [24]:
f.columns.values
label2one = {'B':0,'S':1,'X':2}
one2label = {0:'B', 1:'S', 2:'X'}

In [25]:
def normalize(data):
    mu = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    return mu, std, (data-mu)/std

vfunc = np.vectorize(lambda x:label2one[x])


In [32]:
input_labels = ['pitcher','batter','x0','x','y','ax','ay','az','px','pz','sz_top','sz_bot',
             'vx0','vy0','vz0','pfx_x','z0','start_speed','end_speed',
             'break_y','break_angle','break_length','spin_dir','spin_rate']
train_years = [5,6]
dev_years = [7]


train_x = {}
train_y = {}
ctr = 0
for y in train_years:
    f = pd.read_csv(base_dir+"MLB_201{0}/MLB_PitchFX_201{0}_RegularSeason.csv".format(str(y)))
    
    tmp_x = f[input_labels]
    tmp_y = f['umpcall']

    tmp_x = tmp_x.as_matrix()
    tmp_y = tmp_y.as_matrix()
    tmp_y = vfunc(tmp_y)

    if ctr==0:
        ctr=1
        train_x = tmp_x
        train_y = tmp_y
    else:
        print(train_x.shape)
        print(tmp_x.shape)
        train_x = np.concatenate((train_x, tmp_x), axis=0)
        train_y = np.concatenate((train_y, tmp_y), axis=0)
        
f2 = pd.read_csv(base_dir+"MLB_2017/MLB_PitchFX_2017_PostSeason.csv")
dev_x = f2[input_labels]
dev_y = f2['umpcall']




dev_x = dev_x.as_matrix()
dev_y = dev_y.as_matrix()
dev_y = vfunc(dev_y)



(672271, 24)
(703809, 24)


In [33]:
vv = np.vectorize(lambda x: (isinstance(x, float) and math.isnan(x) ))
idx = ~np.logical_or(vv(train_x[:,0]), vv(train_x[:,1]))
train_x = train_x[idx]
train_y = train_y[idx]

In [ ]:
from sklearn.model_selection import train_test_split
train_x, dev_x, train_y, dev_y = train_test_split(data_x, data_y, test_size=0.1)

In [ ]:
np.sum(train_y==2)

# Cross Validation

In [34]:
def criterion(pred, y):
    s = 0;
    for i in range(pred.shape[0]):
        s -= np.log(pred[i,y[i]])
    return s/y.shape[0]

In [35]:
def accuracy(pred, y):
    pred = np.argmax(pred,axis=1)
    if y is None:
        return 0
    return np.sum(pred == y) / y.shape[0]

In [36]:
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator, ClassifierMixin
class kNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, **kwargs):
        self.model = NearestNeighbors(**kwargs)
    def fit(self, data_x, y):
        X = data_x[:,2:].astype(dtype=np.double)
        self.mu, self.std, self.X = normalize(X)
        
        self.y = y
        
        self.model.fit(self.X)
        self.pitchers = {}
        self.batters = {}
        self.matches = {}
        self.ctr_m = {}
        self.ctr_p = {}
        self.ctr_b = {}
        for i in range(data_x.shape[0]):
            p = data_x[i,0]
            b = data_x[i,1]
            if p in self.pitchers:
                self.pitchers[p] += data_x[i,2:]
                self.ctr_p[p] +=1
            else:
                self.pitchers[p] = data_x[i,2:]
                self.ctr_p[p] = 1
            if b in self.batters:
                self.batters[b] += data_x[i,2:]
                self.ctr_b[b] += 1
            else:
                self.batters[b] = data_x[i,2:]
                self.ctr_b[b] = 1
            if p+' '+b in self.matches:
                self.matches[p+' '+b] += data_x[i,2:]
                self.ctr_m[p+' '+b] += 1
            else:
                self.matches[p+' '+b] = data_x[i,2:]
                self.ctr_m[p+' '+b] = 1
            
        self.avg_data = np.mean(self.X, axis=0)
    def predict(self, dev_x, po=4, threshold=1e-5, oridist=100):
        de = np.zeros((dev_x.shape[0], self.X.shape[1]))
        for i in range(dev_x.shape[0]):
            p = dev_x[i,0]
            b = dev_x[i,1]
            if p+' '+b in self.matches:
                de[i,:] = self.matches[p+' '+b]/self.ctr_m[p+' '+b]
            elif p in self.pitchers:
                de[i,:] = self.pitchers[p]/self.ctr_p[p]
            else:
                de[i,:] = self.avg_data
        
        return self.predict_full(de, po, threshold, oridist)
    
    def predict_full(self, dev_x, po=4, threshold=1e-5, oridist=100):
        dist, indx = self.model.kneighbors((dev_x-self.mu)/self.std)
        preds = (1/(oridist**po))*np.ones((dev_x.shape[0],3))
        for jj in range(len(indx)):
            ctr= 3/(oridist**po)
            for i,e in enumerate(indx[jj]):
                dd = dist[jj][i]+ threshold
                ctr += 1/dd**po
                preds[jj,self.y[e]] += 1/dd**po
            preds[jj,:] /= ctr
        return preds
    def score(self, X, y=None):
        pred = self.predict(X)
        if y is None:
            return 0
        return criterion(pred, y)

In [37]:
model = kNNClassifier(n_neighbors=100, algorithm='auto', n_jobs=-1)
model.fit(train_x, train_y)

In [38]:
print("len:{}".format(dev_x.shape[0]))
pred = model.predict(dev_x, po=4)

len:11223


In [39]:
accuracy(pred,dev_y)

0.44987971130713711

In [40]:
criterion(pred,dev_y)

3.9698163426722877

In [19]:
import time
preds = []
for e in [100,200]:
    model = kNNClassifier(n_neighbors=e, algorithm='auto', n_jobs=-1)
    model.fit(train_x, train_y)
    for po in [2,4]:
        for threshold in [1e-5, 1e-1, 10]:
            for oridist in [1,100,10000]:
                start = time.time()
                pred = model.predict(dev_x, po, threshold, oridist)
                
                print("k:{} po:{} th:{} di:{}".format(e, po, threshold, oridist))
                print("loss:{} acc.:{}".format(criterion(pred,dev_y), accuracy(pred, dev_y)))
                end = time.time()
                print("total time: {}s".format(end-start))
                print()


k:100 po:2 th:1e-05 di:1
loss:1.3028881778642405 acc.:0.4679675666042947
total time: 4.611871719360352s

k:100 po:2 th:1e-05 di:100
loss:1.34035715077066 acc.:0.4679675666042947
total time: 4.487550497055054s

k:100 po:2 th:1e-05 di:10000
loss:1.3772880562573064 acc.:0.4679675666042947
total time: 4.4898107051849365s

k:100 po:2 th:0.1 di:1
loss:1.1914650405671579 acc.:0.4714425732869999
total time: 4.713467359542847s

k:100 po:2 th:0.1 di:100
loss:1.2296912686843346 acc.:0.4714425732869999
total time: 4.684508800506592s

k:100 po:2 th:0.1 di:10000
loss:1.266622262621969 acc.:0.4714425732869999
total time: 4.488931655883789s

k:100 po:2 th:10 di:1
loss:1.04778314901694 acc.:0.4722444979060857
total time: 4.669751882553101s

k:100 po:2 th:10 di:100
loss:1.1928234360053507 acc.:0.4722444979060857
total time: 4.692002296447754s

k:100 po:2 th:10 di:10000
loss:1.2301216249828466 acc.:0.4722444979060857
total time: 4.681848049163818s

k:100 po:4 th:1e-05 di:1
loss:1.5131846974271776 acc.:0.

In [ ]:
import time
eps = [50,100,125,200,300]
preds2 = []
for po in [0,1,2,3,4,5,6]:
    pp = []
    for e in eps:
        start = time.time()
        start1 = time.time()
        model = kNNClassifier(n_neighbors=e, algorithm='auto', n_jobs=-1)
        model.fit(train_x, train_y)
        end=time.time()
        print()
        print("fit: {}s".format(end-start))
        pred = model.predict(dev_x, po=po)
        start=end
        end=time.time()
        print("predict: {}s".format(end-start))
        start=end
        pp.append(pred)
        print("nn:{} loss:{} acc.:{}".format(e, criterion(pred,dev_y), accuracy(pred, dev_y)))
        end = time.time()
        print("total time: {}s".format(end-start1))
    preds2.append(pp)

In [ ]:
end = time.time()
end-start

In [ ]:
S_ctr=0
S_wrong = 0

B_ctr=0
B_wrong = 0

X_ctr=0
X_wrong = 0
pred_label = np.argmax(pred, axis=1)
for i in range(len(pred)):
    if dev_y[i]==1:
        if pred[i]==1:
            S_ctr+=1
        else:
            S_wrong+=1
    if dev_y[i]==0:
        if pred[i]==0:
            B_ctr+=1
        else:
            B_wrong+=1
    if dev_y[i]==2:
        if pred[i]==2:
            X_ctr+=1
        else:
            X_wrong+=1

In [ ]:
pred = model.predict(dev_x)
pred

In [ ]:
print("S({},{}) B({},{}) X({},{})".format(S_ctr, S_wrong, B_ctr, B_wrong, X_ctr, X_wrong))

In [ ]:
d={}
for t in train_y:
    if t not in d:
        d[t] = 1
    else:
        d[t] += 1
d

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, data_x, data_y, cv=5)
scores

# Test Models

In [ ]:
from sklearn.model_selection import cross_val_score
model = kNNClassifier(n_neighbors=200, algorithm='auto')
scores = cross_val_score(model, data_x, data_y, cv=8, n_jobs=-1)
scores

In [ ]:
data_x

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100,), max_iter=1000, alpha=1e-4,
                    solver='adam', tol=1e-5,
                    learning_rate_init=.01)
mlp.fit(train_x, train_y)

In [ ]:
pred = mlp.predict(dev_x)

In [ ]:
pred

In [ ]:
np.sum(mlp.predict(dev_x)==dev_y)/len(dev_y)

In [ ]:
len(dev_y)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,50,), max_iter=100, alpha=1e-4,
                    solver='adam', tol=1e-4,
                    learning_rate_init=.001)

scores = cross_val_score(mlp, data_x, data_y, cv=5)
scores